In [2]:
import os
import sys

spark_home = os.environ['SPARK_HOME']
sys.path.insert(0, spark_home + "/python/")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.1
      /_/

Using Python version 2.7.9 (default, Mar  9 2015 16:20:48)
SparkContext available as sc, HiveContext available as sqlContext.


In [10]:
import csv
import StringIO
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

def loadRecord(line):
    """Parse a CSV line"""
    input = StringIO.StringIO(line)
    reader = csv.reader(input)
    row = map(float, reader.next())
    return LabeledPoint(row[-1],row[:-1]) 


In [25]:
chf = open('data/CAhousing.csv','r')
header = chf.next().rstrip("\n").split(",")
for i,j in enumerate(header):
    print "%d: %s" % (i,j)

0: longitude
1: latitude
2: housingMedianAge
3: totalRooms
4: totalBedrooms
5: population
6: households
7: medianIncome
8: medianHouseValue


In [18]:
chrdd = sc.parallelize(chf).map(lambda line: loadRecord(line))
chrdd.persist()
chrdd.first()

LabeledPoint(452600.0, [-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252])

In [19]:
(trainingData, testData) = chrdd.randomSplit([0.7, 0.3])

In [34]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    impurity='variance', minInstancesPerNode=2500)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression tree model:')
print(model.toDebugString())

Test Mean Squared Error = 7928420168.77
Learned regression tree model:
DecisionTreeModel regressor of depth 3 with 9 nodes
  If (feature 7 <= 5.0)
   If (feature 7 <= 3.0542)
    If (feature 1 <= 34.57)
     Predict: 156987.59329779132
    Else (feature 1 > 34.57)
     Predict: 114907.97661233168
   Else (feature 7 > 3.0542)
    If (feature 2 <= 26.0)
     Predict: 186286.10897189064
    Else (feature 2 > 26.0)
     Predict: 225446.19343065695
  Else (feature 7 > 5.0)
   Predict: 328652.08717785554



In [35]:
?DecisionTree.trainRegressor